In [ ]:
"""
演示怎么在训练好的模型上重新加载并运行。
"""

import pandas as pd
from numpy import zeros
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, LSTM, Bidirectional
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
import sys
sys.path.append("../common/")
import diabetes_helper

X, Y, X_train_np, X_test_np, Y_train_np, Y_test_np = diabetes_helper.prepare_diabetes_data_for_task()

model = Sequential()
model.add(Dense(20, input_shape=(8, )))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
plot_model(model, to_file="./model_info/pima_model_structure.png" , show_shapes=True)

# 1. 存取模型。在val_acc最好的情况下把模型存取。这种方式把模型所有的信息都存下来了，包括模型结构，每层的权重，最有函数的选择等。
model_file="./model_info/pima_model"
checkpoint = ModelCheckpoint(model_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history = model.fit(X_train_np, Y_train_np, batch_size=16, epochs=20, verbose=1, validation_split=0.2, callbacks=callbacks_list)
score, accuracy = model.evaluate(X_test_np, Y_test_np, verbose=0)
print("test score : {} test acc : {}".format(score, accuracy))

# 2. load模型继续训练，为什么结果不一样？
loaded_model = load_model(model_file)
score, accuracy = loaded_model.evaluate(X_test_np, Y_test_np, verbose=0)
print("just load from file")
print("test score : {} test acc : {}".format(score, accuracy))

history2 = loaded_model.fit(X_train_np, Y_train_np, batch_size=16, epochs=20, verbose=1, validation_split=0.2)
score, accuracy = loaded_model.evaluate(X_test_np, Y_test_np, verbose=0)
print("test score : {} test acc : {}".format(score, accuracy))
